In [195]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import datetime

from api_keys import api_key

url = f"https://api.nal.usda.gov/ndb/reports/?"




In [196]:
#Creación del formulario para el usuario

user_dict = {
    "First_Name":'',
    "Last_Name": '',
    "Age":'',
    "Sex":'',
    "Weight_in_Kilograms":'',
    "Height_in_meters":''
    }



for key in user_dict:
    user_dict[key] = str(input(f"Please type your {key} "))

activitie = str(input(f"Please type your type of activitie you have: Sedentary (S), Low Active (L), Active (A) or Very Active (V) "))

if user_dict["Sex"] == "Women":
    pregnant_q = str(input(f"Please type if you are pregnant Yes(Y)/ No(N) (Only the letter) "))
else:
    pregnant_q = "N/A"

if pregnant_q == "Y":
    pregnant_a = int(input(f"In wich trimester of pregnancy are you (Only the number) ? "))
else:
    pregnant_a = "N/A"
    
if user_dict["Sex"] == "Women":
    lactation_q = str(input(f"Please type if you are in lactation Yes(Y)/ No(N) (Only the letter)"))
else:
    lactation_q = "N/A"

if lactation_q == "Y":
    lactation_a = int(input(f"Please type if you are inside the 1st or 2nd Semester of lactation (Only put the number) "))
else:
    lactation_a = "N/A"



Please type your First_Name Paulina
Please type your Last_Name Martinez
Please type your Age 30
Please type your Sex Women
Please type your Weight_in_Kilograms 62
Please type your Height_in_meters 1.62
Please type your type of activitie you have: Sedentary (S), Low Active (L), Active (A) or Very Active (V) S
Please type if you are pregnant Yes(Y)/ No(N) (Only the letter) N
Please type if you are in lactation Yes(Y)/ No(N) (Only the letter)N


In [197]:
#Cálculo del IMC actual y el ideal.

IMC_actual = round(float(user_dict["Weight_in_Kilograms"])/(float(user_dict["Height_in_meters"])**2),2)

if (IMC_actual < 24.9 and IMC_actual > 18.5):
    IMC_ideal = IMC_actual    
if IMC_actual > 24.9:
    IMC_ideal = 24.5
if IMC_actual < 18.5:
    IMC_ideal = 19
    
Ideal_Weight = round(IMC_ideal * (float(user_dict["Height_in_meters"])**2),1)

Variance_Weight = round(float(user_dict["Weight_in_Kilograms"]) - float(Ideal_Weight),1)

print(IMC_actual)
print(Ideal_Weight)
print(Variance_Weight)

23.62
62.0
0.0


In [198]:
#Cálculo de la actividad física, para la fórmula de EER

if user_dict["Sex"] == "Men":
    if activitie == "S":
        P_Activitie = 1
    elif activitie == "L":
        P_Activitie = 1.11
    elif activitie == "A":
        P_Activitie = 1.25
    elif activitie == "V":
        P_Activitie = 1.48
else:
    if activitie == "S":
        P_Activitie = 1
    elif activitie == "L":
        P_Activitie = 1.12
    elif activitie == "A":
        P_Activitie = 1.27
    elif activitie == "V":
        P_Activitie = 1.45


#Cálculo del requerimiento de energía diario de una persona.

EER_Men= float(662) - (9.53 * float(user_dict["Age"]))+ float(P_Activitie) *((15.91*float(Ideal_Weight)) + (539.6*float(user_dict["Height_in_meters"]))) 
EER_Women= float(354) - (6.91 * float(user_dict["Age"]))+ float(P_Activitie) *((9.36*float(Ideal_Weight)) + (float(726)*float(user_dict["Height_in_meters"])))

    
if user_dict["Sex"] == "Men":
    Nut_Req_Atnow = round(EER_Men,2)
else:
    Nut_Req_Atnow = round(EER_Women,2)
    if pregnant_q == "Y":
        if pregnant_a == 1:
            Nut_Req_Atnow = Nut_Req_Atnow
        if pregnant_a == 2:
            Nut_Req_Atnow = Nut_Req_Atnow + 340
        if pregnant_a == 3:
            Nut_Req_Atnow = Nut_Req_Atnow + 452
    if lactation_q == "Y":
        if lactation_a == 1:
            Nut_Req_Atnow = Nut_Req_Atnow + 600 - 170
        if lactation_a == 2:
            Nut_Req_Atnow = Nut_Req_Atnow + 400
            

print(f"{Nut_Req_Atnow} Kcal")


1903.14 Kcal


In [199]:
#Distribución de la energía en carbohidratos, grasas y proteina

Carb_Dist_kcal= round(Nut_Req_Atnow * .50,2)
Fat_Dist_kcal= round(Nut_Req_Atnow * .3,2)
Protein_Dist_kcal = round(Nut_Req_Atnow * .2,2)

print(Carb_Dist_kcal)
print(Fat_Dist_kcal)
print(Protein_Dist_kcal)

951.57
570.94
380.63


In [200]:
#Conversión de Kcal a gramos

Carb_Dist_gr= round(Carb_Dist_kcal / float(4),2)
Fat_Dist_gr= round(Fat_Dist_kcal / float(9),2)
Protein_Dist_gr = round(Protein_Dist_kcal / float(4),2)

print(Carb_Dist_gr)
print(Fat_Dist_gr)
print(Protein_Dist_gr)

237.89
63.44
95.16


In [201]:


User_Information=pd.DataFrame({"Name": user_dict["First_Name"],
                              "Last Name": user_dict["Last_Name"],
                               "Age": user_dict["Age"],
                               "Sex": user_dict["Sex"],
                               "Weight (Kg)": user_dict["Weight_in_Kilograms"],
                               "Height (m)": user_dict["Height_in_meters"],
                               "Pregnant": pregnant_q,
                               "Pregnancy Trimester": pregnant_a,
                               "Lactation": lactation_q,
                               "Lactation Semester": lactation_a,
                               "Actual IMC": IMC_actual,
                               "Ideal IMC": IMC_ideal,
                               "Ideal Weight(Kg)": Ideal_Weight,
                               "Physical Activitie": activitie,
                               "Energy Requirement (kcal)": Nut_Req_Atnow,
                               "Carbohydrates Req (kcal)": Carb_Dist_kcal,
                               "Fat Req (kcal)": Fat_Dist_kcal,
                               "Protein Req (kcal)": Protein_Dist_kcal,
                               "Carbohydrates Req (gr)": Carb_Dist_gr,
                               "Fat Req (gr)": Fat_Dist_gr,
                               "Protein Req (gr)": Protein_Dist_gr
                              }, index=[0])

pd.options.display.max_columns=21


User_Information

,Name,Last Name,Age,Sex,Weight (Kg),Height (m),Pregnant,Pregnancy Trimester,Lactation,Lactation Semester,Actual IMC,Ideal IMC,Ideal Weight(Kg),Physical Activitie,Energy Requirement (kcal),Carbohydrates Req (kcal),Fat Req (kcal),Protein Req (kcal),Carbohydrates Req (gr),Fat Req (gr),Protein Req (gr)
0,Paulina,Martinez,30,Women,62,1.62,N,N/A,N,N/A,23.62,23.62,62.0,S,1903.14,951.57,570.94,380.63,237.89,63.44,95.16
